In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

sns.set(style='whitegrid')
myfont = matplotlib.font_manager.FontProperties(family="Droid Sans Fallback")#正常显示中文
# 用来正常显示负号
plt.rcParams['axes.unicode_minus'] = False

# 查询matplotlib支持的中文字体
from matplotlib.font_manager import fontManager
import os
fonts = [font.name for font in fontManager.ttflist if
         os.path.exists(font.fname) and os.stat(font.fname).st_size > 1e6]
for font in fonts:
    print(font)


# **一、导入数据，更改数据类型**
* *数据来源：共有两个数据源，分别来自[Github(canghailan)](https://github.com/canghailan/Wuhan-2019-nCoV)和个人每日统计。Github数据包括国内外所有省份/州每日累计确诊、死亡、治愈人数，2020-02-07后数据从今日头条接口采集，每小时57分自动更新。国家、省级历史数据会根据卫健委数据更新，市级历史数据与卫健委数据有差异（来自介绍），支持数据接口；个人统计则来源于国家/省卫健委官方每日发布数据，主要包含了每日各类人员的增长人数，数据每日更新。*
* *数据处理：主要包括修改数据类型以及字段名称*

## **1. 导入数据，分别来自Github和个人统计**

In [ ]:
# 来自Github，详情请移至https://github.com/canghailan/Wuhan-2019-nCoV，read_csv数据接口即可
ncov_city = pd.read_csv(
    "https://raw.githubusercontent.com/canghailan/Wuhan-2019-nCoV/master/Wuhan-2019-nCoV.csv")
# 来自个人统计，从1/21日截至目前累计确诊、死亡、治愈、疑似人数；每日增加人数以及其中湖北省增加人数，每日更新
ncov_coun = pd.read_excel("../input/2019ncovfrom121/ncov.xlsx",sheet_name="全国数据")


In [ ]:
#最后一次更新时间
ncov_city['date'].max()

In [ ]:
print('ncov_city columns:', ncov_city.columns)
print('ncov_coun columns:', ncov_coun.columns)


## **2、更改数据类型**

In [ ]:
ncov_city.dtypes

In [ ]:
# 将date类型修改为datetime，将城市代码修改为object
ncov_city.date = pd.to_datetime(ncov_city.date)
ncov_city['provinceCode'] = ncov_city['provinceCode'].astype(object)
ncov_city['cityCode'] = ncov_city['cityCode'].astype(object)


## **3、查看是否有空值出现，并对数据作解释**

In [ ]:
ncov_city.info()

In [ ]:
ncov_city.head()

In [ ]:
ncov_city.tail()

* 从数据中看到province字段和city字段包含大量空值，除中国外其他国家province/provinceCode/city/cityCode字段全部缺失，这部分不做处理，国内这四个字段都为空值的则是最新全国累计确诊人数，city字段为空的则是最新每个省份累计确诊人数，所以对这部分数据也不做其它处理。（虽然没遇到过这种数据表示方式，但确实便于统计数据。）

# 二、展示截至目前疫情涉及的国家和城市情况，以及最新数据
* *查询此次疫情所涉及国家，以及国内所涉及省份；*
* *查询截至目前（跟随Github数据更新）最新的确诊、死亡以及治愈人数情况*


## **1. 返回本次疫情所涉及的国家和城市情况**

In [ ]:
countries = ncov_city['country'].unique()

print(countries)
print('\nAll Countries:', len(countries))


In [ ]:
provinces = ncov_city[ncov_city['country'] == '中国']['province'].unique()

print(provinces)
print('\nAll Provinces Of China:', len(provinces)-1)


* 本次疫情感染共涉及205个国家，全国内共涉及34个省份/直辖市（包括台湾、香港、澳门，除去nan）。

## **2、查看截至目前确诊人数最多的二十个省份**

In [ ]:
# 返回截至目前最新的全国各省市累计数据
ncov_new = ncov_city[ncov_city['date'] ==
                     ncov_city['date'].max()][ncov_city['country'] == '中国']
# 全国每日累计数据
ncov_new_coun = ncov_new[ncov_new['province'].isna()]
# 各省份每日数据
ncov_new_prov = ncov_new[ncov_new['province'].notna()][ncov_new['city'].isna()]
# 各城市每日数据
ncov_new_city = ncov_new[ncov_new['city'].notna()]


In [ ]:
ncov_prov_top = ncov_new_prov.sort_values(by="confirmed", ascending=False)[
    :20].sort_values(by="confirmed", ascending=True)

plt.figure(figsize=(12,8))
plt.barh(
    y=ncov_prov_top['province'],
    width=ncov_prov_top['confirmed'],
    height=0.9,
    color='r')
plt.suptitle('全国新型冠状病毒累计确诊人数最多的二十个省份', fontsize=20 , fontproperties=myfont)
plt.yticks(ncov_prov_top['province'], fontsize=10 , fontproperties=myfont)
for a, b in zip(ncov_prov_top['province'], ncov_prov_top['confirmed']):
    plt.text(x=b+100, y=a, s=b, ha='left', va='center', fontsize=10)

plt.grid(b=False)
plt.show()


## **3、截至目前全国所有省份累计确诊、死亡和治愈人数**

In [ ]:
ncov_new_prov[['date', 'province', 'confirmed', 'dead', 'cured']
              ].sort_values(by='confirmed', ascending=False).set_index('date')


* 总结以上，本次疫情涉及国家不在少数，但重灾区依然在中国的湖北省，目前全国31个省份包括港澳台均已出现确诊人员，而且湖北省确诊人数是确诊人数排第二浙江省的几十倍，在死亡人数当中，湖北省占据了绝大部分。

# **三、全国每日确诊、死亡、治愈人数增长情况**
* *查看并展示全国每日累计确诊、死亡和治愈人数变动情况；查看每日重症患者增长情况，并在每日累计确诊人数的基础上展示，查看所占比例；*
* *结合每日新增密切接触人数和解除观察人数查看当前疫情发展动向；*
* *展示每日新增治愈、死亡人数，比较观察增长情况。*

## **1. 全国每日增加确诊人数以、重症人数以及新增疑似人数**

In [ ]:
def ncov_rise(c, d, e, f, g):
    
    plt.figure(figsize=(12,8))
    plt.bar(ncov_coun[c], ncov_coun[d], color='r', width=0.9, label=d)
    plt.bar(ncov_coun[c], ncov_coun[e], color='k', width=0.9, label=e)
    plt.plot(ncov_coun[c], ncov_coun[f], '-', color='#FF7F24', label=f)

    plt.suptitle(g, fontsize=20, fontproperties=myfont)
    plt.legend(loc='upper left', prop = myfont)
    plt.grid(b=False)
    plt.show()


In [ ]:
ncov_rise('日期', '全国新增确诊', '新增重症', '新增疑似', '全国新型冠状病毒每日新增确诊和重症人数')


## 2、全国每日新增密切接触和当日解除观察人数

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(ncov_coun['日期'], ncov_coun['当日密切接触'], 'r-', label='当日密切接触')
plt.plot(ncov_coun['日期'], ncov_coun['当日解除观察'], 'g-', label='当日解除观察')

plt.suptitle('全国新型冠状病毒每日新增密切接触和解除观察人数',fontsize=20,fontproperties=myfont)
plt.legend(loc='upper left',prop = myfont)
plt.grid(b=False)
plt.show()


* 上图展示了每日新增确诊以及其他相关人员数量情况，其中每日确诊人数从2月12日开始就在持续下降（2月12日湖北将临床诊断病例判定为确诊）；新增疑似人数更是从2月5日起就开始持续下降；而且每日重症人员数量从2月19日开始已经实现了0增长，重症人员得消耗更多的医疗资源和医护人员的精力，这对湖北省可以减轻很大一部分压力。
* 但是只看每日确诊增加人数太过片面，所以当日密切接触人员的增长情况也是一项重要指标，可以从图中看出当日密切接触人员的增长速度有所减缓，并且从2月11日开始持续低于每日解除观察人数，目前均已降至五千以下。
* 当日新增密切接触人员从三月中旬开始反弹，主要是由于境外输入带来的密接增加。

## 3、全国每日累计死亡以及确诊人数

In [ ]:
plt.figure(figsize=(12,8))
plt.plot(ncov_coun['日期'], ncov_coun['累计死亡'], 'r-', label='累计死亡')
plt.plot(ncov_coun['日期'], ncov_coun['累计治愈'], 'g-', label='累计治愈')

plt.suptitle('全国新增冠状病毒累计死亡和治愈人数', fontsize=20, fontproperties=myfont)
plt.legend(prop=myfont)
plt.grid(b=False)
plt.show()


## 4、每日滚动死亡率和滚动治愈率
*滚动死亡率=累计死亡人数/累计确诊人数；滚动治愈率=累计治愈人数/累计确诊人数*

In [ ]:
dead = ncov_coun['累计死亡']/ncov_coun['累计确诊']
cured = ncov_coun['累计治愈']/ncov_coun['累计确诊']

plt.figure(figsize=(12,8))
plt.plot(ncov_coun['日期'], dead, 'r-', label='滚动死亡率')
plt.plot(ncov_coun['日期'], cured, 'g-', label='滚动治愈率')
plt.plot(ncov_coun['日期'], [0.03]*len(ncov_coun['日期']),
         'b--', label='百分之三', alpha=0.7)

plt.suptitle('全国新型冠状病毒每日滚动死亡率和滚动治愈率', fontsize=20, fontproperties=myfont)
plt.legend(prop=myfont)
plt.grid(b=False)
plt.show()


* 从上方两张图可以看出来从1月27日开始每日治愈人数和滚动治愈率就在持续上升，并且在2月1日超过了累计死亡人数；而滚动死亡率则是已经超过了3%，有稳定下来的趋势。

## 5、每日剩余患者

In [ ]:
still = ncov_coun['累计确诊']-ncov_coun['累计死亡']-ncov_coun['累计治愈']
today = [ncov_coun['日期'].iloc[-8], still.iloc[-1]]
date = ncov_coun[still == max(still)]['日期']

plt.figure(figsize=(12,8))
plt.plot(ncov_coun['日期'], still, 'ro-', label='剩余患者')

plt.annotate('Max:{0}'.format(max(still)),
             xy=(date, max(still)), xytext=('2020-02-20', 40000),
             arrowprops=dict(facecolor='black', shrink=0.06))
plt.annotate('Today:{0}'.format(today[1]),
             xy=(date, max(still)),
             xytext=(today[0], today[1]-500))
plt.suptitle('全国新型冠状病毒每日剩余患者人数', fontsize=20, fontproperties=myfont)
plt.legend(prop=myfont)
plt.grid(False)
plt.show()


* 从每日现有确诊患者来看，在二月中旬已经达到最多患者数量，而后一直在持续减少。

# 四、对比湖北省和除湖北省外省份情况对比
* *对比湖北省和除湖北省外其它省份的各项人数增长情况*
* *查询外来入境人员确诊情况*

## 1、全国累计确诊人数和湖北省累计确诊人数

In [ ]:
ncov_coun['湖北确诊占比'] = ncov_coun['湖北新增确诊']/ncov_coun['全国新增确诊']
ncov_coun['湖北死亡占比'] = ncov_coun['湖北新增死亡']/ncov_coun['全国新增死亡']
ncov_coun['湖北治愈占比'] = ncov_coun['湖北新增治愈']/ncov_coun['全国新增治愈']

plt.figure(figsize=(12,8))
plt.plot(ncov_coun['日期'], ncov_coun['湖北确诊占比'], 'k-', label='湖北确诊占比')
plt.plot(ncov_coun['日期'], ncov_coun['湖北死亡占比'], 'r-', label='湖北死亡占比')
plt.plot(ncov_coun['日期'], ncov_coun['湖北治愈占比'], 'g-', label='湖北治愈占比')

plt.suptitle('全国新型冠状病毒湖北确诊和死亡和治愈占比', fontsize=20, fontproperties=myfont)
plt.legend(prop=myfont)
plt.grid(False)
plt.show()


* 从图中可以看出来，湖北省每日新增确诊人数持续占全国的40%以上，随着时间的推移从三月初开始这个比例就在不断下降；
* 而且湖北省每日新增死亡人数持续占全国新增死亡人数的80%以上，而且大部分占比在90%以上；
* 从治愈占比来看，从2月5日开始湖北治愈占比就在持续上升，向援助湖北省一线的四万多名医护人员致敬。

## 2、湖北省新增确诊人数对比其他省份新增治愈人数

In [ ]:
def other_province(arg1, arg2, arg3, arg4, arg5):
    other = ncov_coun[arg1]-ncov_coun[arg3]  # 得到除湖北外其他省份新增确诊人数

    plt.figure(figsize=(12,8))
    plt.plot(ncov_coun['日期'], ncov_coun[arg3], 'k-', label=arg2)
    plt.plot(ncov_coun['日期'], other, 'g-', label=arg4)

    plt.suptitle(arg5, fontsize=20, fontproperties=myfont)
    plt.legend(prop=myfont)
    plt.grid(False)
    plt.show()


In [ ]:
other_province('全国新增确诊','湖北新增确诊','湖北新增确诊','除湖北外新增确诊','全国新型冠状病毒湖北新增确诊和除湖北外新增确诊人数')

* 湖北省2月4日当日新增确诊人数达到最高点，随后每日增长人数都在隔天减少，增长速度相比2月4日之前已有明显下降；除湖北省外其他省份确诊人数总和从2月3日下降十三天后被打断（山东一监狱当天增加确诊200人），随后除湖北外其他省份确诊人数仍持续下降。

## 3、湖北省新增死亡人数对比其他省份新增死亡人数

In [ ]:
other_province(
    '全国新增死亡',
    '湖北新增死亡',
    '湖北新增死亡',
    '除湖北外新增死亡',
    '全国新型冠状病毒湖北新增确诊和除湖北外新增确诊人数')


* 湖北省每日新增死亡人数依然在持续增加，从二月下旬开始已呈下降趋势；除湖北省外其他省份每日新增死亡人数的总和除了2月13日外并未出现很大的起伏。

## 4、湖北省新增治愈人数对比其他省份新增治愈人数

In [ ]:
other_province(
    '全国新增治愈',
    '湖北新增治愈',
    '湖北新增治愈',
    '除湖北外新增治愈',
    '全国新型冠状病毒湖北新增确诊和除湖北外新增确诊人数')


## 5、中国境外输入人员累计确诊情况

In [ ]:
other_country = ncov_city[ncov_city['city'] == '境外输入'][[
    'date', 'province', 'city', 'confirmed', 'cured', 'dead']]
other_country[other_country['date'] == other_country['date'].max()].set_index(
    'date').sort_values('confirmed', ascending=False)


In [ ]:
# 入境人员确诊数据从3月1日开始统计
ncov_other_coun = ncov_coun[ncov_coun['日期'] >=
                            '2020-03-01'][['日期', '全国新增确诊', '新增入境', '累计入境']]
ncov_other_coun['国内新增确诊'] = ncov_other_coun['全国新增确诊']-ncov_other_coun['新增入境']

plt.figure(figsize=(12, 8))
plt.plot(ncov_other_coun['日期'], ncov_other_coun['累计入境'])

plt.suptitle('全国新型冠状病毒境外输入累计确诊人数', fontsize=20, fontproperties=myfont)
plt.grid(False)
plt.show()


In [ ]:
plt.figure(figsize=(12, 8))

plt.plot(
    ncov_other_coun['日期'],
    ncov_other_coun['国内新增确诊'],
    label='国内新增确诊')
plt.plot(
    ncov_other_coun['日期'],
    ncov_other_coun['新增入境'],
    label='境外新增确诊')

plt.suptitle('全国新型冠状病毒本地和境外输入确诊人数', fontsize=20, fontproperties=myfont)
for a, b, c in zip(
        ncov_other_coun['日期'], ncov_other_coun['国内新增确诊'], ncov_other_coun['新增入境']):
    plt.text(x=a, y=b+3, s=b, ha='center', va='bottom', fontsize=10)
    plt.text(x=a, y=c+3, s=c, ha='center', va='bottom', fontsize=10)
plt.legend(prop=myfont)
plt.grid(False)
plt.show()


* 每日新增治愈人数从增长曲线上来看依然在持续上升，湖北省从2月5日开始每日治愈人数大幅度上升，这和源源不断投入的人力和医疗资源有很大的关系；相比之下，除湖北省之外其他省份每日治愈人数也在平缓上升，就目前来看都在往好的方向发展。
* 从3月1日开始从境外涌入的确诊人数开始快速增加，目前国外形势严峻，大量在外留学务工人士想要回国寻求更好的医疗资源，目前已增至八百多号人,主要分布在北京市。
* 从3月13日开始本地新增确诊人数已经少于境外输入确诊人数，新冠病毒所带来的危机已有国内传播转向外来入境人员。

# 五、查询和自己相关的城市确诊人数变化情况
* *可惜Github数据源中并没有每日确诊增加数据，所以只能返回从1月20日开始所查询省市每日累计确诊情况*

In [ ]:
# 返回从1月20日开始中国所有的数据
ncov_copy = ncov_city[ncov_city['date'] >=
                      '2020-01-20'][ncov_city['country'] == '中国'][ncov_city['province'].notna()]
ncov_copy = ncov_copy[['date', 'province', 'city', 'confirmed']]
# 返回各省份数据
ncov_copy_prov = ncov_copy[ncov_copy['city'].isna()]
# 返回各城市数据
ncov_copy_city = ncov_copy[ncov_copy['city'].notna()]

def prov_city(prov='湖北省', city=['武汉市']):
    ncov_prov = ncov_copy_prov[ncov_copy_prov['province'] == prov]
    plt.figure(figsize=(18, 8))
    plt.plot(ncov_prov['date'], ncov_prov['confirmed'], 'r-', label=prov)

    plt.suptitle('{0}新型冠状病毒每日累计确诊情况'.format(prov), fontsize=20, fontproperties=myfont)
    plt.legend(prop=myfont)
    plt.grid(False)

    plt.show()

    for c in city:
        ncov_city_ = ncov_copy_city[ncov_copy_city['city'] == c]
        plt.figure(figsize=(18, 8))
        plt.plot(ncov_city_['date'], ncov_city_['confirmed'], 'r-', label=c)

        plt.suptitle('{0}新型冠状病毒每日累计确诊情况'.format(c), fontsize=20, fontproperties=myfont)
        plt.legend(prop=myfont)
        plt.grid(False)

        plt.show()


In [ ]:
prov_city('北京市', ['西安市', '保定市', '海淀区'])


* 所关心的几个城市累计人数都有所下降并稳定下来，每日基本没有增加确诊人数，希望一切尽快好起来！

# 六、结论
* *本次疫情感染共涉及205个国家，全国内共涉及34个省份/直辖市（包括台湾、香港、澳门）；*
* *2月1日作为新型冠状病毒的分水岭，在这之前每日增长人数和累计确诊人数都是在平缓上升，在这之后疫情全面爆发，但已在2月12日到达顶峰，随后就在持续减少；*
* *新型冠状病毒的滚动死亡率已经上升到3%，滚动治愈率已有趋于稳定的趋势，目前已达到90%以上；*
* *湖北省作为本次疫情的重灾区从2月12过后每日新增确诊人数就在持续下降了，除湖北外其他省份也呈下降趋势；*
* *从3月1日开始涌入部分境外人员，所幸人数并不多，而且下机后就被隔离处理，得到了很好的控制，并没有因此导致本地确诊人数增加。*